# Massively Parallel ABC-SMC

Efficient use of hardware acceleration platforms such as GPUs have revolutionized the domain of machine learning, especially deep learning. Simulation-based inference methods used in a wide variety of scientific disciples, which are currently run on CPUs, have a similar potential of improvement when mapped to parallelized paradigm of hardware acceleration platforms. In this work we propose a new simulation-based inference algorithm that is uniquely developed to exploit parallelism. 

## Model

In the work, we consider a stocastic epidemology model used to predict and analyze the case data of COVID-19. The model has an intractible likelihood function, hence we need to use likelihood free inference.

For training the model, we use data from the Johns Hopkins Dataset. (https://github.com/CSSEGISandData/COVID-19). We use the daily case data of Confirmed, Recevered and Deaths for a certain country. The code below preprocesses the data. The data preprocessing code was adapted from https://github.com/imdevskp/covid_19_jhu_data_web_scrap_and_cleaning

In [ ]:
# Prerequisites for downloading and processing data
!pip install wget
!pip install countryinfo
!nvidia-smi
!pip install gputil

In [ ]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime, timedelta
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# to download data
import wget
# numerical analysis
import numpy as np

In [ ]:
# GPU profiling

import GPUtil
from threading import Thread
import time

class Monitor(Thread):
    def __init__(self, delay):
        super(Monitor, self).__init__()
        self.stopped = False
        self.delay = delay # Time between calls to GPUtil
        self.start()

    def run(self):
        while not self.stopped:
            GPUtil.showUtilization()
            time.sleep(self.delay)

    def stop(self):
        self.stopped = True

In [ ]:
# remove all existing csv files
! rm *.csv

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# download files
for url in urls:
    filename = wget.download(url)

In [ ]:
# dataset
conf_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recv_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [ ]:
# extract dates
dates = conf_df.columns[4:]

# melt dataframes into longer format
# ==================================
conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

recv_df_long = recv_df_long[recv_df_long['Country/Region']!='Canada']

print(conf_df_long.shape)
print(deaths_df_long.shape)
print(recv_df_long.shape)

In [ ]:
full_table = pd.merge(left=conf_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recv_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

In [ ]:
# Convert to proper date format
full_table['Date'] = pd.to_datetime(full_table['Date'])

# fill na with 0
full_table['Recovered'] = full_table['Recovered'].fillna(0)

# convert to int datatype
full_table['Recovered'] = full_table['Recovered'].astype('int')

In [ ]:
import tensorflow as tf
from countryinfo import CountryInfo

def get_country_data(country):
  """
  Method to obtain case data for a certain country.
  Input(String): name of country
  Output(tf.constant): case data of country from the day of 100 confirmed cases
  """
  country_data = (full_table[full_table['Country/Region'] == country])[['Confirmed', 'Recovered', 'Deaths']]
  Confirmed = tf.constant(country_data[['Confirmed']].values, dtype=tf.float32)
  Recovered = tf.constant(country_data[['Recovered']].values, dtype=tf.float32)
  Deaths = tf.constant(country_data[['Deaths']].values, dtype=tf.float32)
  country_data_full = tf.squeeze(tf.stack([Confirmed, Recovered, Deaths]))
  # find start point, when sum(A,R,D) = 100
  start_date = tf.reduce_min(tf.where(tf.reduce_sum(country_data_full, axis=0) >= 100))
  return country_data_full[:, start_date:]

In [ ]:
# Select data of a certain country

COUNTRY = 'Italy'

country_data = get_country_data(COUNTRY)

TRAIN_DAYS = 120
TEST_DAYS = 150

population = CountryInfo(COUNTRY).population()
country_data_train = country_data[:,:TRAIN_DAYS]
country_data_test = country_data[:,:TEST_DAYS]

In [ ]:
# imports

import numpy as np
import time as time

import tensorflow_probability as tfp
import matplotlib.pyplot as plt

tfd = tfp.distributions
# for GPU
tf.device("/GPU:0")

## Model class and other helper functions used for plotting results

In [ ]:
# Some important constants

num_samples = 100000 # level of parallelism

max_runs = 1000 # max runs to run the algorithms for

NUM_TRIALS = 10

In [ ]:
# Model and other helper functions

def plot_summary(summary):
    x_axis = np.arange(country_data_test.shape[1])
    # x_axis = np.arange(country_data_test[:,TRAIN_DAYS:].shape[1])
    fig, axs = plt.subplots(3)
    fig.tight_layout()

    A_mean = tfp.stats.percentile(summary[:, 0, :], 50, axis=0)
    A_min = tfp.stats.percentile(summary[:, 0, :], 0.5, axis=0)
    A_max = tfp.stats.percentile(summary[:, 0, :], 99.5, axis=0)
    R_mean = tfp.stats.percentile(summary[:, 1, :], 50, axis=0)
    R_min = tfp.stats.percentile(summary[:, 1, :], 0.5, axis=0)
    R_max = tfp.stats.percentile(summary[:, 1, :], 99.5, axis=0)
    D_mean = tfp.stats.percentile(summary[:, 2, :], 50, axis=0)
    D_min = tfp.stats.percentile(summary[:, 2, :], 0.5, axis=0)
    D_max = tfp.stats.percentile(summary[:, 2, :], 99.5, axis=0)

    axs[0].set(ylabel="Active", yscale="log")
    axs[0].vlines(TRAIN_DAYS,0,np.max(country_data_test[0, :]), linestyles='dashed')
    axs[0].plot(x_axis, country_data_test[0, :], "black")
    axs[0].plot(x_axis, A_mean, "b")
    axs[0].fill_between(
        x_axis,
        A_min,
        A_max,
        alpha=0.3,
        color="b",
    )
    

    axs[1].set(ylabel="Recovered", yscale="log")
    axs[1].vlines(TRAIN_DAYS,0,np.max(country_data_test[1, :]), linestyles='dashed')
    axs[1].plot(x_axis, country_data_test[1, :], "black")
    axs[1].plot(x_axis, R_mean, "g")
    axs[1].fill_between(
        x_axis,
        R_min,
        R_max,
        alpha=0.3,
        color="g",
    )

    axs[2].set(ylabel="Deaths", yscale="log")
    axs[2].vlines(TRAIN_DAYS,0,np.max(country_data_test[2, :]), linestyles='dashed')
    axs[2].plot(x_axis, country_data_test[2, :], "black")
    axs[2].plot(x_axis, D_mean, "r")
    axs[2].fill_between(
        x_axis,
        D_min,
        D_max,
        alpha=0.3,
        color="r",
    )

    fig.savefig("plots.png", dpi=400)


class Covid19Model:
    def __init__(self, num_days, num_samples, population, A_0, R_0, D_0):
        self.num_days = (
            num_days  
        )
        self.num_samples = (
            num_samples 
        )
        self.P = tf.ones(num_samples) * population
        self.A_0 = tf.ones(num_samples) * A_0
        self.R_0 = tf.ones(num_samples) * R_0
        self.D_0 = tf.ones(num_samples) * D_0
        self.param_vector = tf.transpose(
            tfd.Uniform(
                tf.constant([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
                tf.constant([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
            ).sample(self.num_samples)
        )
        self.S_store = (
            self.P - 2 * self.param_vector[7] * self.A_0 - (self.A_0 + self.R_0 + self.D_0)
        )
        self.I_store = 2 * self.param_vector[7] * self.A_0
        self.A_store = self.A_0
        self.R_store = self.R_0
        self.D_store = self.D_0
        self.Ru_store = tf.zeros(self.num_samples)
        self.summary = tf.zeros([self.num_days, 3, self.num_samples])
        self.summary = tf.tensor_scatter_nd_add(
            self.summary,
            [[0, 0], [0, 1], [0, 2]],
            tf.stack([self.A_store, self.R_store, self.D_store]),
        )
        # order of params: alpha_0, alpha, n, beta, gamma, delta, eta, kappa

        self.nu = tf.constant(
            [
                [-1, 1, 0, 0, 0, 0],
                [0, -1, 1, 0, 0, 0],
                [0, 0, -1, 1, 0, 0],
                [0, 0, -1, 0, 1, 0],
                [0, -1, 0, 0, 0, 1],
            ],
            dtype=tf.float32,
        )

    def one_day(self, i, s, S_store, I_store, A_store, R_store, D_store, Ru_store):
        alpha_t = self.param_vector[0] + (
            100 * self.param_vector[1]
            / (
                tf.constant(1.0)
                + tf.pow(A_store + R_store + D_store, 2 * self.param_vector[2])
            )
        )
        h_1 = (S_store * I_store / self.P) * alpha_t
        h_2 = I_store * self.param_vector[4]
        h_3 = A_store * self.param_vector[3]
        h_4 = A_store * self.param_vector[5]
        h_5 = I_store * self.param_vector[6] * self.param_vector[3]
        h = tf.stack([h_1, h_2, h_3, h_4, h_5])
        Y_store = tf.clip_by_value(
            tf.math.floor(tfd.Normal(loc=h, scale=tf.sqrt(h)).sample()), 0.0, self.P
        )
        S = tf.clip_by_value(
            S_store + tf.tensordot(self.nu[:, 0], Y_store, axes=1), 0.0, self.P
        )
        I = tf.clip_by_value(
            I_store + tf.tensordot(self.nu[:, 1], Y_store, axes=1), 0.0, self.P
        )
        A = tf.clip_by_value(
            A_store + tf.tensordot(self.nu[:, 2], Y_store, axes=1), 0.0, self.P
        )
        R = tf.clip_by_value(
            R_store + tf.tensordot(self.nu[:, 3], Y_store, axes=1), 0.0, self.P
        )
        D = tf.clip_by_value(
            D_store + tf.tensordot(self.nu[:, 4], Y_store, axes=1), 0.0, self.P
        )
        Ru = tf.clip_by_value(
            Ru_store + tf.tensordot(self.nu[:, 5], Y_store, axes=1), 0.0, self.P
        )
        s = tf.tensor_scatter_nd_add(s, [[i, 0], [i, 1], [i, 2]], tf.stack([A, R, D]))
        (S_store, I_store, A_store, R_store, D_store, Ru_store) = (S, I, A, R, D, Ru)
        return tf.add(i, 1), s, S_store, I_store, A_store, R_store, D_store, Ru_store

    def summary_statistic(self):
        i = tf.constant(1)
        (
            r,
            self.summary,
            self.S_store,
            self.I_store,
            self.A_store,
            self.R_store,
            self.D_store,
            self.Ru_store,
        ) = tf.while_loop(
            lambda i, *_: i < self.num_days,
            self.one_day,
            [
                i,
                self.summary,
                self.S_store,
                self.I_store,
                self.A_store,
                self.R_store,
                self.D_store,
                self.Ru_store,
            ],
            parallel_iterations=1,
        )
        summary = self.summary
        self.reset()
        return summary

    def reset(self):
        self.Y_store = tf.ones(
            [5, self.num_samples], dtype=tf.float32
        )  # !!changed_by_AT!! * -float('inf'))
        self.S_store = (
            self.P - 2 * self.param_vector[7] * self.A_0 - (self.A_0 + self.R_0 + self.D_0)
        )
        self.I_store = 2 * self.param_vector[7] * self.A_0
        self.A_store = self.A_0
        self.R_store = self.R_0
        self.D_store = self.D_0
        self.Ru_store = tf.zeros(self.num_samples)
        self.summary = tf.zeros([self.num_days, 3, self.num_samples])
        self.summary = tf.tensor_scatter_nd_add(
            self.summary,
            [[0, 0], [0, 1], [0, 2]],
            tf.stack([self.A_store, self.R_store, self.D_store]),
        )
        self.param_vector = tf.transpose(
            tfd.Uniform(
                tf.constant([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
                tf.constant([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
            ).sample(self.num_samples)
        )


def collect_summaries(param_vector):
    model = Covid19Model(
        num_days=TEST_DAYS,
        num_samples=param_vector.shape[0],
        population=tf.constant(60.36e6, dtype=tf.float32),
        A_0=country_data_test[0, 0],
        R_0=country_data_test[1, 0],
        D_0=country_data_test[2, 0],
    )
    model.param_vector = tf.transpose(param_vector)
    print(model.param_vector.shape)
    summary = tf.transpose(model.summary_statistic(), perm=[2, 1, 0])
    return summary

In [ ]:
# GPU-optimized simulation code for epidemology model

@tf.function(experimental_compile=True)
def build_graph(tolerance, param_vector):
    num_days = tf.cast(country_data_train.shape[1], tf.int32)
    P = tf.ones(num_samples) * population
    A_0 = tf.ones(num_samples) * country_data_train[0, 0]
    R_0 = tf.ones(num_samples) * country_data_train[1, 0]
    D_0 = tf.ones(num_samples) * country_data_train[2, 0]

    summary = tf.zeros([num_days, 3, num_samples])

    nu = tf.constant(
        [
            [-1, 1, 0, 0, 0, 0],
            [0, -1, 1, 0, 0, 0],
            [0, 0, -1, 1, 0, 0],
            [0, 0, -1, 0, 1, 0],
            [0, -1, 0, 0, 0, 1],
        ],
        dtype=tf.float32,
    )

    S_store = P - 2 * param_vector[7] * A_0 - (A_0 + R_0 + D_0)
    I_store = 2 * param_vector[7] * A_0
    A_store = A_0
    R_store = R_0
    D_store = D_0
    Ru_store = tf.zeros(num_samples)

    summary = tf.tensor_scatter_nd_add(
        summary, [[0, 0], [0, 1], [0, 2]], tf.stack([A_store, R_store, D_store])
    )

    def body(i, s, S, I, A, R, D, Ru):
        U = A + R + D
        alpha_t = param_vector[0] + (
            100 * param_vector[1] / (tf.constant(1.0) + tf.pow(U, 2 * param_vector[2]))
        )
        h_1 = (S * I / P) * alpha_t
        h_2 = I * param_vector[4]
        h_3 = A * param_vector[3]
        h_4 = A * param_vector[5]
        h_5 = I * param_vector[6] * param_vector[3]
        h = tf.stack([h_1, h_2, h_3, h_4, h_5])
        Y_store = tf.clip_by_value(
            tf.math.floor(tfd.Normal(loc=h, scale=tf.sqrt(h)).sample()), 0.0, P
        )

        m = tf.matmul(tf.transpose(nu), Y_store)

        S = tf.clip_by_value(S + m[0, :], 0.0, P)
        I = tf.clip_by_value(I + m[1, :], 0.0, P)
        A = tf.clip_by_value(A + m[2, :], 0.0, P)
        R = tf.clip_by_value(R + m[3, :], 0.0, P)
        D = tf.clip_by_value(D + m[4, :], 0.0, P)
        Ru = tf.clip_by_value(Ru + m[5, :], 0.0, P)

        s = tf.tensor_scatter_nd_add(s, [[i, 0], [i, 1], [i, 2]], tf.stack([A, R, D]))

        return i + 1, s, S, I, A, R, D, Ru

    init_idx = tf.zeros([], dtype=tf.int32) + 1
    i, summary, *_ = tf.while_loop(
        lambda i, *_: i < num_days,
        body,
        [init_idx, summary, S_store, I_store, A_store, R_store, D_store, Ru_store],
    )

    t_summary = tf.transpose(summary, perm=[2, 1, 0])
    # Normalized Distances
    distances = tf.norm(
        tf.broadcast_to(
            country_data_train / tf.reduce_max(country_data_train,axis=1, keepdims=True),
            tf.constant(
                [num_samples, country_data_train.shape[0], country_data_train.shape[1]],
                dtype=tf.int32,
            ) ,
        )
        - t_summary / tf.reduce_max(country_data_train,axis=1, keepdims=True),
        axis=2,
    )

    # # Un-normalized Distances
    # distances = tf.norm(
    #     tf.broadcast_to(
    #         country_data_train,
    #         tf.constant(
    #             [num_samples, country_data_train.shape[0], country_data_train.shape[1]],
    #             dtype=tf.int32,
    #         ) ,
    #     )
    #     - t_summary,
    #     axis=2,
    # )

    reduced_distances = tf.reduce_sum(distances, axis=1)
    acceptance_vector = reduced_distances <= tolerance
    num_accepted_samples = tf.reduce_sum(
        tf.cast(acceptance_vector, dtype=tf.float32), name="num_accepted_samples"
    )
    return num_accepted_samples, acceptance_vector, param_vector, reduced_distances

## Baseline Inference Method: P-ABC-SMC MCMC

As a baseline, we consider the state-of-the-art Adaptive Approximate Bayesian Computation with Sequential Monte Carlo (ABC-SMC) inference with MCMC tuned steps. This is the inference method used by the original authours of the epidemology model.
The algorithm is parallelized to obtain the best possible performance on GPUs.

In [ ]:
## MCMC version

NUM_ADAPT_STEPS = 0.1 * max_runs
per_trial_tolerances_mcmc = []
per_trial_runs_mcmc = []
for _ in range(NUM_TRIALS):
  # Time it
  initial_tolerance = 1e9
  target_tolerance = 0.1
  alpha = 0.5
  print("Running...")
  samples_target = 1000
  samples_collected = 0
  num_runs = 0
  start_time = time.time()
  tolerance = initial_tolerance
  prev_stage_samples = []
  prev_stage_distances = []
  stage = 0
  step_size_multiplier = 1
  step_size = 0.1
  num_accepted_samples = 0
  acceptance_ratio = 1
  per_stage_tolerances_mcmc = []
  per_stage_runs_mcmc = []
  
  # # Performance Profiling
  # sim_time = 0

  while tolerance >= target_tolerance and num_runs <= max_runs: # and acceptance_ratio > 0.02:
      accepted_param_vector = []
      accepted_distances = []
      samples_collected = 0
      for _ in range(max_runs):
          # print(prev_stage_samples)
          if stage == 0: 
              prev_stage_samples = tfd.Uniform(
                  tf.constant([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
                  tf.constant([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
              ).sample(num_samples)
              resampled_params = tf.transpose(prev_stage_samples)
              # stage += 1

          else:
              # resmaple params from last stage and do MCMC acceptance
              resampled_params = tf.transpose(
                  tf.gather(
                      prev_stage_samples,
                      tf.random.categorical([tolerance - prev_stage_distances], num_samples)[0],
                      axis=0,
                  )
              )

          # Metropolis-Hastings Acceptance
          
          if num_runs < NUM_ADAPT_STEPS:
            # step size tuning
            step_size = tf.constant(step_size * step_size_multiplier, dtype=tf.float32)

          # proposed_param_vector = tf.clip_by_value(
          #     tfd.Normal(resampled_params, scale=step_size).sample(), 0, 1
          # )

          # Truncated Gaussian Random Walk
          proposed_param_vector = tfd.TruncatedNormal(loc=resampled_params, scale=step_size, low=0., high=1.).sample()

          fwd_log_prob = tf.reduce_sum(tfd.TruncatedNormal(loc=resampled_params, scale=step_size, low=0., high=1.).log_prob(proposed_param_vector), axis=0)
          rev_log_prob = tf.reduce_sum(tfd.TruncatedNormal(loc=proposed_param_vector, scale=step_size, low=0., high=1.).log_prob(resampled_params), axis=0)
          mcmc_prob = tf.clip_by_value(tf.exp(fwd_log_prob + rev_log_prob), 0, 1)
          random_nos = tf.random.uniform( [num_samples], minval=0, maxval=1)
          mcmc_acceptance_vector = mcmc_prob >= random_nos
          mcmc_rejectance_vector = mcmc_prob < random_nos

          # # Acceptance ratio old version
          # acceptance_ratio = (acceptance_ratio + tf.math.count_nonzero(mcmc_acceptance_vector, dtype=tf.float32)/mcmc_acceptance_vector.shape[0]) / 2

          target_acceptance_ratio = tf.constant(0.234, dtype=tf.float32)

          if num_runs < NUM_ADAPT_STEPS:
            # Compute update to step size
            step_size_multiplier = tf.constant(tf.math.exp((acceptance_ratio - target_acceptance_ratio) /((1.0 - target_acceptance_ratio) * (num_accepted_samples + 1))), dtype=tf.float32)

          param_vector = tf.transpose(tf.concat([tf.boolean_mask(tf.transpose(proposed_param_vector), mcmc_acceptance_vector), tf.boolean_mask(tf.transpose(resampled_params), mcmc_rejectance_vector)], axis=0))

          # Perform one batch of simulation        
          # sim_time_accumulator = time.time() # for profiling
          num_accepted_samples, acceptance_vector, param_vector, distances = build_graph(
              tolerance,
              param_vector,
          )
          # sim_time += time.time() - sim_time_accumulator # for profiling
          # Acceptance ratio new version
          acceptance_ratio = (acceptance_ratio + num_accepted_samples/num_samples) / 2

          # Post processing
          samples_collected += num_accepted_samples
          samples_blah = tf.boolean_mask(tf.transpose(param_vector), acceptance_vector)
          samples_distances = tf.boolean_mask(distances, acceptance_vector)
          for i in range(len(samples_blah)):
              accepted_param_vector.append(samples_blah[i])
              accepted_distances.append(samples_distances[i])
          num_runs += 1
          if samples_collected >= samples_target:
              if tolerance <= target_tolerance or num_runs >= max_runs - 1: 
                  accepted_param_vector = tf.stack(accepted_param_vector)
                  accepted_distances = tf.stack(accepted_distances)
                  break
              prev_stage_distances, accepted_indices = tf.math.top_k(
                  -1 * tf.stack(accepted_distances), int(samples_target * alpha),
              )
              prev_stage_distances = -prev_stage_distances
              prev_stage_samples = tf.gather(
                  tf.stack(accepted_param_vector), accepted_indices
              )
              tolerance = tf.math.top_k(prev_stage_distances)[0]
              stage += 1
              break
      print(f"stage: {stage}, tolerance: {tolerance}, number of runs: {num_runs}, acceptance ratio: {acceptance_ratio}")
      per_stage_tolerances_mcmc.append(tolerance)
      per_stage_runs_mcmc.append(num_runs)
  per_trial_tolerances_mcmc.append(per_stage_tolerances_mcmc)
  per_trial_runs_mcmc.append(per_stage_runs_mcmc)
  stages_mcmc = stage+1
  end_time = time.time()
  print("Completed in {0:.3f} seconds\n".format(end_time - start_time))
  print(f"Samples collected: {samples_collected}")
  print(f"Number of runs: {num_runs}")
  print(
      "Time per run: {0:.3f} milliseconds\n".format(
          1e3 * (end_time - start_time) / num_runs
      )
  )
  # print(
  #     "Time spent in simulation: {0:.3f} milliseconds\n".format(
  #         1e3 * (sim_time) / num_runs
  #     )
  # )
  print(f"Final tolerance: {tolerance[0]}")

  # Plot results
  output_summary = collect_summaries(tf.constant(tf.stack(accepted_param_vector)))
  plot_summary(output_summary)


## Proposed Inference Method: P-ABC-SMC BDSS

In this proposed method, the MCMC based step sized tuning is replaced by step sizes sampled from a Beta distribution, whose shape is modified by the progress of the ABC-SMC process.

In [ ]:
per_trial_tolerances_prop = []
per_trial_runs_prop = []
for trial in range(NUM_TRIALS):
  print(f'Trial number {trial + 1}')
  # Time it
  initial_tolerance = 1e9
  target_tolerance = 0.5
  alpha = 0.5
  print("Running...")
  # max_runs = 50
  samples_target = 1000
  samples_collected = 0
  num_runs = 0
  start_time = time.time()
  tolerance = initial_tolerance
  prev_stage_samples = []
  prev_stage_distances = []
  per_stage_tolerances_prop = []
  per_stage_runs_prop = []
  stage = 0

  # # Performance Profiling
  # sim_time = 0

  while tolerance >= target_tolerance and num_runs <= max_runs:
      accepted_param_vector = []
      accepted_distances = []
      samples_collected = 0
      for _ in range(max_runs):
          
          if stage == 1:
            initial_tolerance = tolerance
          
          if stage == 0:
            prev_stage_samples = tfd.Uniform(
                tf.constant([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
                tf.constant([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]),
            ).sample(num_samples)
            resampled_params = tf.transpose(prev_stage_samples)
          else:
            resampled_params = tf.transpose(
                tf.gather(
                    prev_stage_samples,
                    tf.random.categorical([tolerance - prev_stage_distances], num_samples)[0],
                    axis=0,
                )
            )

          # non-MCMC acceptance
          
          # Original BDSS tuning
          step_size = tfp.distributions.Beta(tolerance/initial_tolerance, 2 * (stage + 1))
          
          # # Mode Concentration reparameterization
          # mode = tolerance/initial_tolerance
          # concentration = 1 * (stage + 1) # 10
          # alpha_param = mode * (concentration - 2) + 1
          # beta = (1 - mode) * (concentration - 2) + 1
          # step_size = tfp.distributions.Beta(alpha_param, beta)

          # # Mean Variance reparameterization
          # mean = tolerance/initial_tolerance
          # variance = 1/(stage+1)
          # alpha_param = mean * variance
          # beta = (1 - mean) * variance
          # step_size = tfp.distributions.Beta(alpha_param, beta)

          # Truncated Gaussian Random Walk Perturbation
          param_vector = tfd.TruncatedNormal(loc=resampled_params, scale=tf.clip_by_value(tf.squeeze(step_size.sample(resampled_params.shape)), 1e-30, 1), low=0.0, high=1.0).sample()
          
          # sim_time_accumulator = time.time() # for profiling          

          num_accepted_samples, acceptance_vector, param_vector, distances = build_graph(
              tolerance,
              param_vector,
          )

          # sim_time += time.time() - sim_time_accumulator # for profiling

          samples_collected += num_accepted_samples
          samples_blah = tf.boolean_mask(tf.transpose(param_vector), acceptance_vector)
          samples_distances = tf.boolean_mask(distances, acceptance_vector)
          for i in range(len(samples_blah)):
              accepted_param_vector.append(samples_blah[i])
              accepted_distances.append(samples_distances[i])
          num_runs += 1
          if samples_collected >= samples_target:
              if tolerance <= target_tolerance or num_runs >= max_runs - 1:
                  accepted_param_vector = tf.stack(accepted_param_vector)
                  accepted_distances = tf.stack(accepted_distances)
                  break
              prev_stage_distances, accepted_indices = tf.math.top_k(
                  -1 * tf.stack(accepted_distances), int(samples_target * alpha),
              )
              prev_stage_distances = -prev_stage_distances
              prev_stage_samples = tf.gather(
                  tf.stack(accepted_param_vector), accepted_indices
              )
              tolerance = tf.math.top_k(prev_stage_distances)[0]
              stage += 1
              break
      print(f"stage: {stage}, tolerance: {tolerance.numpy()[0]}, number of runs: {num_runs}")
      per_stage_tolerances_prop.append(tolerance)
      per_stage_runs_prop.append(num_runs)
  per_trial_tolerances_prop.append(per_stage_tolerances_prop)
  per_trial_runs_prop.append(per_stage_runs_prop)
  stages_prop = stage+1
  end_time = time.time()
  # print(accepted_param_vector)
  print("Completed in {0:.3f} seconds\n".format(end_time - start_time))
  print(f"Samples collected: {samples_collected}")
  print(f"Number of runs: {num_runs}")
  print(
      "Time per run: {0:.3f} milliseconds\n".format(
          1e3 * (end_time - start_time) / num_runs
      )
  )
  # print(
  #     "Time spent in simulation: {0:.3f} milliseconds\n".format(
  #         1e3 * (sim_time) / num_runs
  #     )
  # )
  print(f"Final tolerance: {tolerance[0]}")

  output_summary = collect_summaries(tf.constant(tf.stack(accepted_param_vector)))
  plot_summary(output_summary)


In [ ]:
!mkdir plot_data

In [ ]:
SAVE_DATA = True

plt.figure(dpi=400)
for trial in range(NUM_TRIALS):
  plt.plot(np.arange(len(per_trial_tolerances_mcmc[trial])),np.squeeze(np.array(per_trial_tolerances_mcmc[trial])), 'b')
  plt.plot(np.arange(len(per_trial_tolerances_prop[trial])),np.squeeze(np.array(per_trial_tolerances_prop[trial])), 'r')
  if SAVE_DATA:
    np.save(f'plot_data/per_trial_tolerances_mcmc{trial}', np.squeeze(np.array(per_trial_tolerances_mcmc[trial])))
    np.save(f'plot_data/per_trial_tolerances_prop{trial}', np.squeeze(np.array(per_trial_tolerances_prop[trial])))

plt.plot(np.arange(len(per_trial_tolerances_mcmc[trial])),np.squeeze(np.array(per_trial_tolerances_mcmc[trial])), 'b', label='P-ABC-SMC MCMC')
plt.plot(np.arange(len(per_trial_tolerances_prop[trial])),np.squeeze(np.array(per_trial_tolerances_prop[trial])), 'r', label='P-ABC-SMC BDSS')

plt.grid()
plt.legend()
plt.xlabel('Number of stages')
plt.ylabel('Tolerance')

In [ ]:
SAVE_DATA = True

plt.figure(dpi=400)
for trial in range(NUM_TRIALS):
  plt.plot(np.arange(len(per_trial_runs_mcmc[trial])),np.squeeze(np.array(per_trial_runs_mcmc[trial])), 'b')
  plt.plot(np.arange(len(per_trial_runs_prop[trial])),np.squeeze(np.array(per_trial_runs_prop[trial])), 'r')
  if SAVE_DATA:
    np.save(f'plot_data/per_trial_runs_mcmc{trial}', np.squeeze(np.array(per_trial_runs_mcmc[trial])))
    np.save(f'plot_data/per_trial_runs_prop{trial}', np.squeeze(np.array(per_trial_runs_prop[trial])))

plt.plot(np.arange(len(per_trial_runs_mcmc[trial])),np.squeeze(np.array(per_trial_runs_mcmc[trial])), 'b', label='ABC-SMC MCMC')
plt.plot(np.arange(len(per_trial_runs_prop[trial])),np.squeeze(np.array(per_trial_runs_prop[trial])), 'r', label='ABC-SMC NEW')

plt.grid()
plt.legend()
plt.xlabel('Number of stages')
plt.ylabel('Number of simulation runs')

In [ ]:
plt.figure(dpi=300)
plt.ylim([0,1000000])
# plt.xlim([0,1000])
# plt.xscale('log')
# plt.yscale('log')
for trial in range(NUM_TRIALS):
  plt.plot(np.squeeze(np.array(per_trial_runs_mcmc[trial])), np.squeeze(np.array(per_trial_tolerances_mcmc[trial])), 'b')
  plt.plot(np.squeeze(np.array(per_trial_runs_prop[trial])),np.squeeze(np.array(per_trial_tolerances_prop[trial])), 'r')

plt.plot(np.squeeze(np.array(per_trial_runs_mcmc[trial])), np.squeeze(np.array(per_trial_tolerances_mcmc[trial])), 'b', label='ABC-SMC MCMC')
plt.plot(np.squeeze(np.array(per_trial_runs_prop[trial])),np.squeeze(np.array(per_trial_tolerances_prop[trial])), 'r', label='ABC-SMC NEW')

plt.grid()
plt.legend()
plt.xlabel('Number of simulation runs')
plt.ylabel('Tolerance')

In [ ]:
!zip -r plot_data.zip plot_data/